In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
import joblib
import time
warnings.filterwarnings('ignore')

# ML
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import TimeSeriesSplit
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier

# Paths
BASE_DIR = Path(r'c:\Users\Acer\Desktop\Forex-Signal-App')
DATA_DIR = BASE_DIR / 'data'
MODEL_DIR = BASE_DIR / 'models' / 'signal_generator_v13'
MODEL_DIR.mkdir(parents=True, exist_ok=True)

# GPU Check
import torch
GPU_AVAILABLE = torch.cuda.is_available()

# Trading Parameters
TP_PIPS = 15
SL_PIPS = 10
FORWARD_BARS = 60

print("="*70)
print("🚀 FOREX SIGNAL GENERATOR V13")
print("   TP/SL Based Labeling + Strong Regularization")
print("="*70)
print(f"✓ GPU Available: {GPU_AVAILABLE}")
print(f"✓ TP: {TP_PIPS} pips, SL: {SL_PIPS} pips")
print(f"✓ Forward window: {FORWARD_BARS} bars")
print(f"✓ Model Directory: {MODEL_DIR}")

🚀 FOREX SIGNAL GENERATOR V13
   TP/SL Based Labeling + Strong Regularization
✓ GPU Available: True
✓ TP: 15 pips, SL: 10 pips
✓ Forward window: 60 bars
✓ Model Directory: c:\Users\Acer\Desktop\Forex-Signal-App\models\signal_generator_v13


## 1. Load Data

In [3]:
# Load Data
train_df = pd.read_csv(DATA_DIR / 'EUR_USD_1min.csv')
test_df = pd.read_csv(DATA_DIR / 'EUR_USD_test.csv')

for df in [train_df, test_df]:
    if 'timestamp' in df.columns:
        df.rename(columns={'timestamp': 'time'}, inplace=True)
    df['time'] = pd.to_datetime(df['time'])

print(f"Train: {len(train_df):,} rows")
print(f"Test: {len(test_df):,} rows")
print(f"Train period: {train_df['time'].min()} to {train_df['time'].max()}")
print(f"Test period: {test_df['time'].min()} to {test_df['time'].max()}")

Train: 1,859,492 rows
Test: 296,778 rows
Train period: 2019-12-31 16:00:00+00:00 to 2024-12-30 16:00:00+00:00
Test period: 2024-12-31 16:00:00+00:00 to 2025-10-17 06:11:00+00:00


## 2. TP/SL Based Labeling (BUY-only)

In [4]:
def create_tpsl_labels(df, tp_pips=15, sl_pips=10, forward_bars=60):
    """
    V13 TP/SL Based Labeling for BUY signals:
    - Label = 1 (WIN): TP hit before SL
    - Label = 0 (LOSE): SL hit before TP
    - Label = -1 (NO_TRADE): Neither hit within forward_bars
    """
    df = df.copy()
    tp_price = tp_pips * 0.0001
    sl_price = sl_pips * 0.0001
    
    labels = []
    
    for i in range(len(df)):
        if i + forward_bars >= len(df):
            labels.append(-1)  # Not enough future data
            continue
            
        entry_price = df.iloc[i]['close']
        tp_level = entry_price + tp_price
        sl_level = entry_price - sl_price
        
        # Look forward
        future = df.iloc[i+1:i+1+forward_bars]
        
        tp_hit_idx = None
        sl_hit_idx = None
        
        # Find first TP hit
        tp_hits = future[future['high'] >= tp_level]
        if len(tp_hits) > 0:
            tp_hit_idx = tp_hits.index[0]
        
        # Find first SL hit
        sl_hits = future[future['low'] <= sl_level]
        if len(sl_hits) > 0:
            sl_hit_idx = sl_hits.index[0]
        
        # Determine label
        if tp_hit_idx is not None and sl_hit_idx is not None:
            if tp_hit_idx < sl_hit_idx:
                labels.append(1)  # WIN - TP first
            else:
                labels.append(0)  # LOSE - SL first
        elif tp_hit_idx is not None:
            labels.append(1)  # WIN - only TP hit
        elif sl_hit_idx is not None:
            labels.append(0)  # LOSE - only SL hit
        else:
            labels.append(-1)  # NO_TRADE - neither hit
    
    df['signal'] = labels
    return df

print("Creating TP/SL based labels...")
print("This may take a few minutes...")

start = time.time()
train_df = create_tpsl_labels(train_df, TP_PIPS, SL_PIPS, FORWARD_BARS)
test_df = create_tpsl_labels(test_df, TP_PIPS, SL_PIPS, FORWARD_BARS)
print(f"✓ Labels created in {time.time()-start:.1f}s")

# Filter only WIN/LOSE (remove NO_TRADE)
train_labeled = train_df[train_df['signal'] != -1].copy()
test_labeled = test_df[test_df['signal'] != -1].copy()

print(f"\n📊 Label Distribution:")
print(f"Train: {len(train_labeled):,} samples")
print(f"  WIN (TP hit): {(train_labeled['signal']==1).sum():,} ({(train_labeled['signal']==1).mean()*100:.1f}%)")
print(f"  LOSE (SL hit): {(train_labeled['signal']==0).sum():,} ({(train_labeled['signal']==0).mean()*100:.1f}%)")

print(f"\nTest: {len(test_labeled):,} samples")
print(f"  WIN (TP hit): {(test_labeled['signal']==1).sum():,} ({(test_labeled['signal']==1).mean()*100:.1f}%)")
print(f"  LOSE (SL hit): {(test_labeled['signal']==0).sum():,} ({(test_labeled['signal']==0).mean()*100:.1f}%)")

Creating TP/SL based labels...
This may take a few minutes...
✓ Labels created in 1312.9s

📊 Label Distribution:
Train: 601,304 samples
  WIN (TP hit): 191,049 (31.8%)
  LOSE (SL hit): 410,255 (68.2%)

Test: 121,253 samples
  WIN (TP hit): 40,811 (33.7%)
  LOSE (SL hit): 80,442 (66.3%)


## 3. Feature Engineering

In [5]:
def add_features_v13(df):
    """V13 Feature Engineering - Same proven features"""
    df = df.copy()
    
    # Time Features
    df['hour'] = df['time'].dt.hour
    df['day_of_week'] = df['time'].dt.dayofweek
    df['is_london'] = ((df['hour'] >= 8) & (df['hour'] < 16)).astype(int)
    df['is_ny'] = ((df['hour'] >= 13) & (df['hour'] < 21)).astype(int)
    df['is_overlap'] = ((df['hour'] >= 13) & (df['hour'] < 16)).astype(int)
    
    # Moving Averages
    for p in [5, 10, 20, 50, 200]:
        df[f'sma_{p}'] = df['close'].rolling(p).mean()
        df[f'ema_{p}'] = df['close'].ewm(span=p, adjust=False).mean()
    
    # RSI
    delta = df['close'].diff()
    gain = delta.where(delta > 0, 0).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / (loss + 1e-10)
    df['rsi'] = 100 - (100 / (1 + rs))
    
    # MACD
    ema12 = df['close'].ewm(span=12).mean()
    ema26 = df['close'].ewm(span=26).mean()
    df['macd'] = ema12 - ema26
    df['macd_signal'] = df['macd'].ewm(span=9).mean()
    df['macd_hist'] = df['macd'] - df['macd_signal']
    
    # Bollinger Bands
    df['bb_mid'] = df['close'].rolling(20).mean()
    bb_std = df['close'].rolling(20).std()
    df['bb_upper'] = df['bb_mid'] + 2 * bb_std
    df['bb_lower'] = df['bb_mid'] - 2 * bb_std
    df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / (df['bb_mid'] + 1e-10)
    df['bb_position'] = (df['close'] - df['bb_lower']) / (df['bb_upper'] - df['bb_lower'] + 1e-10)
    
    # ATR
    tr = pd.concat([
        abs(df['high'] - df['low']),
        abs(df['high'] - df['close'].shift()),
        abs(df['low'] - df['close'].shift())
    ], axis=1).max(axis=1)
    df['atr'] = tr.rolling(14).mean()
    df['atr_ratio'] = df['atr'] / (df['close'] + 1e-10)
    
    # Stochastic
    low_14 = df['low'].rolling(14).min()
    high_14 = df['high'].rolling(14).max()
    df['stoch_k'] = 100 * (df['close'] - low_14) / (high_14 - low_14 + 1e-10)
    df['stoch_d'] = df['stoch_k'].rolling(3).mean()
    
    # Price patterns
    df['body'] = df['close'] - df['open']
    df['body_pct'] = df['body'] / (df['open'] + 1e-10)
    df['upper_wick'] = df['high'] - df[['open', 'close']].max(axis=1)
    df['lower_wick'] = df[['open', 'close']].min(axis=1) - df['low']
    df['range'] = df['high'] - df['low']
    df['range_atr'] = df['range'] / (df['atr'] + 1e-10)
    
    # Momentum
    for p in [5, 10, 20]:
        df[f'momentum_{p}'] = df['close'] - df['close'].shift(p)
        df[f'roc_{p}'] = df['close'].pct_change(p) * 100
    
    # Trend
    df['trend_5'] = (df['close'] > df['sma_5']).astype(int)
    df['trend_20'] = (df['close'] > df['sma_20']).astype(int)
    df['trend_50'] = (df['close'] > df['sma_50']).astype(int)
    df['trend_strength'] = df['trend_5'] + df['trend_20'] + df['trend_50']
    
    # MA distances
    df['dist_sma20'] = (df['close'] - df['sma_20']) / (df['sma_20'] + 1e-10) * 100
    df['dist_sma50'] = (df['close'] - df['sma_50']) / (df['sma_50'] + 1e-10) * 100
    
    # Volatility
    df['volatility_10'] = df['close'].rolling(10).std()
    df['volatility_20'] = df['close'].rolling(20).std()
    
    # Volume
    if 'volume' in df.columns:
        df['volume_sma'] = df['volume'].rolling(20).mean()
        df['volume_ratio'] = df['volume'] / (df['volume_sma'] + 1e-10)
    
    return df.dropna()

print("Adding features...")
train_labeled = add_features_v13(train_labeled)
test_labeled = add_features_v13(test_labeled)

print(f"✓ Train: {len(train_labeled):,} samples")
print(f"✓ Test: {len(test_labeled):,} samples")

Adding features...
✓ Train: 601,105 samples
✓ Test: 121,054 samples
✓ Train: 601,105 samples
✓ Test: 121,054 samples


## 4. Prepare Data

In [6]:
# Define feature columns
exclude_cols = ['time', 'signal', 'open', 'high', 'low', 'close', 'volume', 'tick_volume']
feature_cols = [c for c in train_labeled.columns if c not in exclude_cols]

# Only numeric columns
feature_cols = [c for c in feature_cols if train_labeled[c].dtype in ['float64', 'int64', 'float32', 'int32']]

print(f"📊 Features: {len(feature_cols)}")

X_train = train_labeled[feature_cols].values
y_train = train_labeled['signal'].values
X_test = test_labeled[feature_cols].values
y_test = test_labeled['signal'].values

# Scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"✓ X_train: {X_train_scaled.shape}")
print(f"✓ X_test: {X_test_scaled.shape}")
print(f"✓ y_train WIN ratio: {y_train.mean()*100:.1f}%")
print(f"✓ y_test WIN ratio: {y_test.mean()*100:.1f}%")

📊 Features: 50
✓ X_train: (601105, 50)
✓ X_test: (121054, 50)
✓ y_train WIN ratio: 31.8%
✓ y_test WIN ratio: 33.7%
✓ X_train: (601105, 50)
✓ X_test: (121054, 50)
✓ y_train WIN ratio: 31.8%
✓ y_test WIN ratio: 33.7%


## 5. Train V13 Models (Strong Regularization)

In [7]:
print("="*70)
print("🚀 TRAINING V13 MODELS")
print("   With Strong Regularization to Prevent Overfitting")
print("="*70)

models = {}

# XGBoost - Strong Regularization
print("\n📊 Training XGBoost...")
models['xgb1'] = xgb.XGBClassifier(
    n_estimators=300,
    max_depth=4,           # Shallow trees
    learning_rate=0.01,    # Low learning rate
    subsample=0.6,         # Strong subsampling
    colsample_bytree=0.6,
    reg_alpha=1.0,         # Strong L1
    reg_lambda=5.0,        # Strong L2
    min_child_weight=10,   # More samples per leaf
    gamma=0.5,             # Pruning
    tree_method='hist',
    device='cuda',
    random_state=42,
    early_stopping_rounds=50,
    eval_metric='logloss'
)

models['xgb2'] = xgb.XGBClassifier(
    n_estimators=300,
    max_depth=3,
    learning_rate=0.02,
    subsample=0.5,
    colsample_bytree=0.5,
    reg_alpha=2.0,
    reg_lambda=10.0,
    min_child_weight=20,
    gamma=1.0,
    tree_method='hist',
    device='cuda',
    random_state=43,
    early_stopping_rounds=50,
    eval_metric='logloss'
)

# LightGBM - Strong Regularization
print("📊 Training LightGBM...")
models['lgb1'] = lgb.LGBMClassifier(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.01,
    subsample=0.6,
    colsample_bytree=0.6,
    reg_alpha=1.0,
    reg_lambda=5.0,
    min_child_samples=50,
    device='gpu',
    random_state=42,
    verbose=-1
)

models['lgb2'] = lgb.LGBMClassifier(
    n_estimators=300,
    max_depth=3,
    learning_rate=0.02,
    subsample=0.5,
    colsample_bytree=0.5,
    reg_alpha=2.0,
    reg_lambda=10.0,
    min_child_samples=100,
    device='gpu',
    random_state=43,
    verbose=-1
)

# CatBoost - Strong Regularization
print("📊 Training CatBoost...")
models['cat1'] = CatBoostClassifier(
    iterations=300,
    depth=4,
    learning_rate=0.01,
    l2_leaf_reg=10,
    min_data_in_leaf=50,
    task_type='GPU',
    random_state=42,
    verbose=0
)

models['cat2'] = CatBoostClassifier(
    iterations=300,
    depth=3,
    learning_rate=0.02,
    l2_leaf_reg=20,
    min_data_in_leaf=100,
    task_type='GPU',
    random_state=43,
    verbose=0
)

print(f"\n✓ Created {len(models)} models")

🚀 TRAINING V13 MODELS
   With Strong Regularization to Prevent Overfitting

📊 Training XGBoost...
📊 Training LightGBM...
📊 Training CatBoost...

✓ Created 6 models


In [8]:
# Train models
trained_models = {}
accuracies = {}

for name, model in models.items():
    print(f"\n🔧 Training {name}...")
    start = time.time()
    
    try:
        if 'xgb' in name:
            # XGBoost with eval set for early stopping
            model.fit(
                X_train_scaled, y_train,
                eval_set=[(X_test_scaled, y_test)],
                verbose=False
            )
        else:
            model.fit(X_train_scaled, y_train)
        
        train_time = time.time() - start
        
        # Evaluate
        train_pred = model.predict(X_train_scaled)
        test_pred = model.predict(X_test_scaled)
        
        train_acc = accuracy_score(y_train, train_pred)
        test_acc = accuracy_score(y_test, test_pred)
        gap = train_acc - test_acc
        
        trained_models[name] = model
        accuracies[name] = test_acc
        
        overfit = "⚠️" if gap > 0.05 else "✓"
        print(f"   Train: {train_acc*100:.1f}%, Test: {test_acc*100:.1f}%, Gap: {gap*100:+.1f}% {overfit}")
        print(f"   Time: {train_time:.1f}s")
        
    except Exception as e:
        print(f"   ❌ Error: {e}")

print(f"\n✓ Trained {len(trained_models)} models")


🔧 Training xgb1...
   Train: 69.9%, Test: 68.3%, Gap: +1.6% ✓
   Time: 24.1s

🔧 Training xgb2...
   Train: 69.9%, Test: 68.3%, Gap: +1.6% ✓
   Time: 24.1s

🔧 Training xgb2...
   Train: 70.0%, Test: 68.4%, Gap: +1.6% ✓
   Time: 7.7s

🔧 Training lgb1...
   Train: 70.0%, Test: 68.4%, Gap: +1.6% ✓
   Time: 7.7s

🔧 Training lgb1...
   Train: 69.9%, Test: 68.3%, Gap: +1.6% ✓
   Time: 10.9s

🔧 Training lgb2...
   Train: 69.9%, Test: 68.3%, Gap: +1.6% ✓
   Time: 10.9s

🔧 Training lgb2...
   Train: 70.0%, Test: 68.3%, Gap: +1.7% ✓
   Time: 6.5s

🔧 Training cat1...
   Train: 70.0%, Test: 68.3%, Gap: +1.7% ✓
   Time: 6.5s

🔧 Training cat1...
   Train: 69.6%, Test: 67.9%, Gap: +1.6% ✓
   Time: 34.9s

🔧 Training cat2...
   Train: 69.6%, Test: 67.9%, Gap: +1.6% ✓
   Time: 34.9s

🔧 Training cat2...
   Train: 69.8%, Test: 68.2%, Gap: +1.6% ✓
   Time: 91.0s

✓ Trained 6 models
   Train: 69.8%, Test: 68.2%, Gap: +1.6% ✓
   Time: 91.0s

✓ Trained 6 models


## 6. Ensemble & Evaluation

In [9]:
print("="*70)
print("📊 ENSEMBLE EVALUATION")
print("="*70)

# Calculate weights based on accuracy
total_acc = sum(accuracies.values())
weights = {name: acc/total_acc for name, acc in accuracies.items()}

print("\nModel Weights:")
for name, w in sorted(weights.items(), key=lambda x: -x[1]):
    print(f"  {name}: {w:.3f}")

# Get predictions
proba_dict = {}
preds_dict = {}
for name, model in trained_models.items():
    proba_dict[name] = model.predict_proba(X_test_scaled)
    preds_dict[name] = model.predict(X_test_scaled)

# Weighted ensemble probability
ensemble_proba = np.zeros((len(X_test_scaled), 2))
for name, proba in proba_dict.items():
    ensemble_proba += weights[name] * proba

# WIN probability
win_prob = ensemble_proba[:, 1] * 100

# Agreement bonus
all_preds = np.array([preds_dict[name] for name in trained_models.keys()])
win_votes = np.sum(all_preds == 1, axis=0)

all_agree = win_votes == len(trained_models)
strong_agree = win_votes >= (len(trained_models) - 1)
majority = win_votes >= (len(trained_models) // 2 + 1)

confidence = win_prob.copy()
confidence[all_agree] = np.minimum(confidence[all_agree] + 5, 100)
confidence[strong_agree & ~all_agree] = np.minimum(confidence[strong_agree & ~all_agree] + 3, 100)
confidence[majority & ~strong_agree] = np.minimum(confidence[majority & ~strong_agree] + 1, 100)

print(f"\nConfidence Distribution:")
print(f"  >= 90%: {(confidence >= 90).sum():,}")
print(f"  >= 80%: {(confidence >= 80).sum():,}")
print(f"  >= 70%: {(confidence >= 70).sum():,}")
print(f"  >= 60%: {(confidence >= 60).sum():,}")

📊 ENSEMBLE EVALUATION

Model Weights:
  xgb2: 0.167
  xgb1: 0.167
  lgb2: 0.167
  lgb1: 0.167
  cat2: 0.167
  cat1: 0.166

Confidence Distribution:
  >= 90%: 0
  >= 80%: 0
  >= 70%: 26
  >= 60%: 1,595

Confidence Distribution:
  >= 90%: 0
  >= 80%: 0
  >= 70%: 26
  >= 60%: 1,595


In [10]:
# Results by threshold
print("\n📈 V13 Results by Threshold:")
print("-"*80)
print(f"{'Threshold':>10} | {'Signals':>10} | {'Wins':>10} | {'Win Rate':>10} | {'PF':>10}")
print("-"*80)

v13_results = {}
for threshold in [50, 55, 60, 65, 70, 75, 80, 85, 90]:
    mask = confidence >= threshold
    if mask.sum() > 0:
        signals = mask.sum()
        wins = y_test[mask].sum()
        losses = signals - wins
        win_rate = wins / signals * 100
        pf = (wins * TP_PIPS) / max(1, losses * SL_PIPS)
        
        v13_results[threshold] = {
            'signals': signals,
            'wins': wins,
            'losses': losses,
            'win_rate': win_rate,
            'pf': pf
        }
        
        marker = "🎯" if win_rate >= 50 else ""
        print(f"{threshold:>8}%+ | {signals:>10,} | {wins:>10,} | {win_rate:>9.1f}% | {pf:>9.2f} {marker}")
    else:
        print(f"{threshold:>8}%+ | {'0':>10} | {'-':>10} | {'-':>10} | {'-':>10}")


📈 V13 Results by Threshold:
--------------------------------------------------------------------------------
 Threshold |    Signals |       Wins |   Win Rate |         PF
--------------------------------------------------------------------------------
      50%+ |      7,432 |      4,912 |      66.1% |      2.92 🎯
      55%+ |      5,111 |      3,509 |      68.7% |      3.29 🎯
      60%+ |      1,595 |      1,247 |      78.2% |      5.38 🎯
      65%+ |        107 |         99 |      92.5% |     18.56 🎯
      70%+ |         26 |         25 |      96.2% |     37.50 🎯
      75%+ |          0 |          - |          - |          -
      80%+ |          0 |          - |          - |          -
      85%+ |          0 |          - |          - |          -
      90%+ |          0 |          - |          - |          -


## 7. Overfit Check

In [11]:
# Check train vs test
print("="*70)
print("🔍 OVERFIT CHECK: Train vs Test")
print("="*70)

# Get train predictions
train_proba_dict = {}
train_preds_dict = {}
for name, model in trained_models.items():
    train_proba_dict[name] = model.predict_proba(X_train_scaled)
    train_preds_dict[name] = model.predict(X_train_scaled)

train_ensemble = np.zeros((len(X_train_scaled), 2))
for name, proba in train_proba_dict.items():
    train_ensemble += weights[name] * proba

train_win_prob = train_ensemble[:, 1] * 100

# Agreement
train_all_preds = np.array([train_preds_dict[name] for name in trained_models.keys()])
train_votes = np.sum(train_all_preds == 1, axis=0)
train_all_agree = train_votes == len(trained_models)
train_strong = train_votes >= (len(trained_models) - 1)
train_majority = train_votes >= (len(trained_models) // 2 + 1)

train_conf = train_win_prob.copy()
train_conf[train_all_agree] = np.minimum(train_conf[train_all_agree] + 5, 100)
train_conf[train_strong & ~train_all_agree] = np.minimum(train_conf[train_strong & ~train_all_agree] + 3, 100)
train_conf[train_majority & ~train_strong] = np.minimum(train_conf[train_majority & ~train_strong] + 1, 100)

print(f"\n{'Threshold':>10} | {'Train Sig':>10} | {'Train WR':>10} | {'Test Sig':>10} | {'Test WR':>10} | {'Gap':>8}")
print("-"*80)

for threshold in [50, 60, 70, 80, 90]:
    train_mask = train_conf >= threshold
    test_mask = confidence >= threshold
    
    train_sig = train_mask.sum()
    test_sig = test_mask.sum()
    
    train_wr = y_train[train_mask].mean() * 100 if train_sig > 0 else 0
    test_wr = y_test[test_mask].mean() * 100 if test_sig > 0 else 0
    
    gap = train_wr - test_wr
    status = "✓" if abs(gap) < 5 else ("⚡" if abs(gap) < 10 else "⚠️")
    
    print(f"{threshold:>8}%+ | {train_sig:>10,} | {train_wr:>9.1f}% | {test_sig:>10,} | {test_wr:>9.1f}% | {gap:>+7.1f}% {status}")

🔍 OVERFIT CHECK: Train vs Test

 Threshold |  Train Sig |   Train WR |   Test Sig |    Test WR |      Gap
--------------------------------------------------------------------------------
      50%+ |     38,017 |      63.4% |      7,432 |      66.1% |    -2.7% ✓
      60%+ |      8,495 |      72.5% |      1,595 |      78.2% |    -5.7% ⚡
      70%+ |        142 |      96.5% |         26 |      96.2% |    +0.3% ✓
      80%+ |          0 |       0.0% |          0 |       0.0% |    +0.0% ✓
      90%+ |          0 |       0.0% |          0 |       0.0% |    +0.0% ✓

 Threshold |  Train Sig |   Train WR |   Test Sig |    Test WR |      Gap
--------------------------------------------------------------------------------
      50%+ |     38,017 |      63.4% |      7,432 |      66.1% |    -2.7% ✓
      60%+ |      8,495 |      72.5% |      1,595 |      78.2% |    -5.7% ⚡
      70%+ |        142 |      96.5% |         26 |      96.2% |    +0.3% ✓
      80%+ |          0 |       0.0% |          0

## 8. Save V13 Models

In [12]:
print("="*70)
print("💾 SAVING V13 MODELS")
print("="*70)

# Save models
for name, model in trained_models.items():
    joblib.dump(model, MODEL_DIR / f'{name}_v13.joblib')
    print(f"✓ {name}_v13.joblib")

# Save scaler
joblib.dump(scaler, MODEL_DIR / 'scaler_v13.joblib')
print(f"✓ scaler_v13.joblib")

# Save features
joblib.dump(feature_cols, MODEL_DIR / 'feature_cols_v13.joblib')
print(f"✓ feature_cols_v13.joblib")

# Save weights
joblib.dump(weights, MODEL_DIR / 'weights_v13.joblib')
print(f"✓ weights_v13.joblib")

# Save config
config = {
    'version': 'v13',
    'labeling': 'TP/SL Based',
    'tp_pips': TP_PIPS,
    'sl_pips': SL_PIPS,
    'forward_bars': FORWARD_BARS,
    'models': list(trained_models.keys()),
    'features': len(feature_cols),
    'accuracies': accuracies,
    'results': v13_results
}
joblib.dump(config, MODEL_DIR / 'config_v13.joblib')
print(f"✓ config_v13.joblib")

print(f"\n✅ All V13 models saved to: {MODEL_DIR}")

💾 SAVING V13 MODELS
✓ xgb1_v13.joblib
✓ xgb2_v13.joblib
✓ lgb1_v13.joblib
✓ lgb2_v13.joblib
✓ cat1_v13.joblib
✓ cat2_v13.joblib
✓ scaler_v13.joblib
✓ feature_cols_v13.joblib
✓ weights_v13.joblib
✓ config_v13.joblib

✅ All V13 models saved to: c:\Users\Acer\Desktop\Forex-Signal-App\models\signal_generator_v13


## 9. Final Summary

In [13]:
print("="*80)
print("🎯 V13 FINAL SUMMARY")
print("="*80)

# Find best threshold
best_th = max(v13_results.keys(), key=lambda x: v13_results[x]['pf'] if v13_results[x]['signals'] >= 100 else 0)
best = v13_results[best_th]

print(f"""
📊 TRAINING SETUP:
   Labeling: TP/SL Based (TP={TP_PIPS} pips, SL={SL_PIPS} pips)
   Models: {len(trained_models)} ({', '.join(trained_models.keys())})
   Features: {len(feature_cols)}
   Regularization: Strong (max_depth=3-4, reg_alpha/lambda high)

📈 BEST RESULT (Threshold {best_th}%+):
   Signals: {best['signals']:,}
   Wins: {best['wins']:,}
   Losses: {best['losses']:,}
   Win Rate: {best['win_rate']:.1f}%
   Profit Factor: {best['pf']:.2f}

💰 EXPECTED PROFIT (1000 trades @ $1/pip):
   Gross Profit: ${best['wins'] * TP_PIPS:,}
   Gross Loss: ${best['losses'] * SL_PIPS:,}
   Net Profit: ${best['wins'] * TP_PIPS - best['losses'] * SL_PIPS:,}
""")

print("="*80)
print("✅ V13 TRAINING COMPLETE!")
print("="*80)

🎯 V13 FINAL SUMMARY

📊 TRAINING SETUP:
   Labeling: TP/SL Based (TP=15 pips, SL=10 pips)
   Models: 6 (xgb1, xgb2, lgb1, lgb2, cat1, cat2)
   Features: 50
   Regularization: Strong (max_depth=3-4, reg_alpha/lambda high)

📈 BEST RESULT (Threshold 65%+):
   Signals: 107
   Wins: 99
   Losses: 8
   Win Rate: 92.5%
   Profit Factor: 18.56

💰 EXPECTED PROFIT (1000 trades @ $1/pip):
   Gross Profit: $1,485
   Gross Loss: $80
   Net Profit: $1,405

✅ V13 TRAINING COMPLETE!
